![IFMG](https://storage.googleapis.com/ifmg/IFMG.png)

---
# Sistemas de Recomendação com kNN (k-Nearest Neighbors)

* **Autor: Prof. Felipe Reis**
* **Data: 2021-12-27**

### Referências

* SpatialKey. Real estate transaction. 2008. Disponível em: https://support.spatialkey.com/spatialkey-sample-csv-data/. Acesso em: 2021-12-27.

* Wikipedia Contributors. *k-nearest neighbors algorithm*. 2021. Disponível em: https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm. Acesso em: 2021-12-27.

* DEDHIA, Heeral. *Movie Recommendation and Rating Prediction using K-Nearest Neighbors*. 2020. Disponível em: https://www.analyticsvidhya.com/blog/2020/08/recommendation-system-k-nearest-neighbors/. Acesso em: 2021-12-27.

* KORSTANJE, Joos. *The k-Nearest Neighbors (kNN) Algorithm in Python*. 2021. Disponível em: https://realpython.com/knn-python/. Acesso em: 2021-12-27.

* SOFIA, Porta. *Recommendation System using kNN*. 2020. Disponível em: https://www.aurigait.com/blog/recommendation-system-using-knn/. Acesso em: 2021-12-27.

* Datagy.IO. *Normalize a Pandas Column or Dataframe (w/ Pandas or sklearn).* Disponível em: https://datagy.io/pandas-normalize-column/. Acesso em: 2021-12-27.

* Scikit-Learn. *KNeighborsClassifier*. 2021. Disponível em: https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html. Acesso em: 2021-12-27.

* Scikit-Learn. *DistanceMetric*. 2021. Disponível em: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.DistanceMetric.html#sklearn.metrics.DistanceMetric. Acesso em: 2021-12-27.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import scipy as sp
from scipy.sparse import csr_matrix

from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MaxAbsScaler

ImportError: this version of pandas is incompatible with numpy < 1.15.4
your numpy version is 1.14.5.
Please upgrade numpy to >= 1.15.4 to use this pandas version

## Leitura da Base de Dados

A base de dados contém 985 transações imobiliárias na área de Sacramento relatadas em um período de cinco dias. A base foi disponibilizada pelo site Sacramento Bee (https://www.sacbee.com/). Os dados foram obtidos no link: https://support.spatialkey.com/spatialkey-sample-csv-data/.

In [ ]:
#load dataframe from csv
imoveis = pd.read_csv('sacramentorealestatetransactions.csv', delimiter=',')

#imprime informações do dataframe
imoveis.info()

In [ ]:
imoveis = pd.read_csv('sacramentorealestatetransactions.csv', delimiter=',')

#remove colunas desnecessárias
imoveis = imoveis.drop('street', axis=1)
imoveis = imoveis.drop('zip', axis=1)
imoveis = imoveis.drop('state', axis=1)
imoveis = imoveis.drop('sale_date', axis=1)

#imprime dataframe 
imoveis

#df.head() #imprime somente primeiras linhas
#df.tail() #imprime somente últimas linhas

#df['city'] #imprime uma coluna

In [ ]:
#obtém informações estatísticas (algumas estatísticas não fazem sentido para o conjunto de dados)
imoveis.describe()

In [ ]:
#exibe um registro em específico
imoveis.iloc[0]

## Produção de Estatísticas

Obter estatísticas é importante para entender a base de dados e as informações do qual iremos produzir conhecimento.

In [ ]:
#exibição de informações estatísica sobre número de quartos por imóveis
#print(np.min(imoveis['beds']))

#conta a quantidade de imoveis pela quantidade de quartos (no ex., de 0 a 8)
imoveis_por_qtd_quarto = np.bincount(imoveis['beds'])
print(imoveis_por_qtd_quarto) 


#plot gráfico para verificar informações
fig = plt.figure()
ax = pd.Series(imoveis['beds']).value_counts().sort_values(ascending=True).plot.barh(width=0.9)
plt.title('Quantidade de Imóveis por Número Quartos')
plt.show()

In [ ]:
hist = imoveis['price'].hist(bins=10)
hist

## Análise de Correlação entre as informações

In [ ]:
#podemos traçar a correlação entre as informações (quanto mais próximo de 1, melhor)
imoveis_corr = imoveis.corr()
print(imoveis_corr)

In [ ]:
#a partir da análise, podemos remover a latitude e longitude do modelo
#latitude e longitude podem ser usadas para filtrar imóveis próximos, utilizando distância euclidiana
#fórmula: https://stackoverflow.com/questions/28994289/calculate-euclidean-distance-with-google-maps-coordinates

imoveis = imoveis.drop('latitude', axis=1)
imoveis = imoveis.drop('longitude', axis=1)

imoveis.head()

In [ ]:
#podemos traçar a correlação entre as informações (quanto mais próximo de 1, melhor)
imoveis_corr = imoveis.corr()
print(imoveis_corr)

## Implementação da Matriz para Correção das Informações

Uma matriz pode ser utilizada para realizar a correlação entre os itens. 

Como a matriz gerada é esparsa (muitos valores zero, não correlacionados), podemos representá-la utilizando uma matriz esparsa.

### Referências:

* GeeksForGeeks. *Sparse Matrix Representations | Set 3 (CSR)*. 2021. 
Disponível em: https://www.geeksforgeeks.org/sparse-matrix-representations-set-3-csr/. Acesso em: 2021-12-27.

* Scipy. *CSR Matrix*. 2021. Disponível em: https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html. Acesso em: 2021-12-27.

In [ ]:
imoveis = imoveis.drop('city', axis=1)
imoveis = imoveis.drop('type', axis=1)

In [ ]:
imoveis.values

In [ ]:
imoveis

In [ ]:
matriz_features = csr_matrix(imoveis.values)
print(matriz_features[0:5]) #imprime alguns valores da matriz de features (correlação)

## Criação do Modelo

Para melhor utilização do modelo, devem ser levadas em consideração as métricas utilizadas para calcular a distância entre elementos.

![distância](https://upload.wikimedia.org/wikipedia/commons/0/08/Manhattan_distance.svg)
*Fonte: https://en.wikipedia.org/wiki/Taxicab_geometry.* 

*Legenda: Distância Euclidiana em verde, Distância Manhattan em azul.*


### Distância Euclidiana

Uma métrica básica é a Distância Euclidiana, também conhecida como distância em linha reta. 

Tal medida, em um plano cartesiano de duas dimensões, é feita pela seguinte fórmula:

$$ d= \sqrt{(x_1 - x_0)^2 + (y_1 - y_0)^2} $$

Essa métrica pode ser generalizada para distâncias não cartesianas e múltiplos elementos, representados por tuplas. Ex.: $P = (p_1, p_2, p_3, ..., p_n)$ e $Q = (q_1, q_2, q_3, ..., q_n)$.

$$ d= \sqrt{(p_1 - q_1)^2 + (p_2 - q_2)^2 + ... + (p_n - q_n)^2} $$


### Distância $L_1$ ou Manhattan (Métrica do Táxi)

A distância $L_1$ é uma métrica para cálculo da distância entre dois pontos, baseado na soma das diferenças absolutas de suas coordenadas. Foi criada por Minkowski como alternativa à distância Euclidiana. Seu nome faz alusão à cidade de Manhattan e à movimentação de táxis em uma cidade. 

A métrica, para pontos $P = (p_1, p_2, p_3, ..., p_n)$ e $Q = (q_1, q_2, q_3, ..., q_n)$, pode ser definida por:

$$ d = |p_1 - q_1| + |p_2 - q_2| + ... + |p_n - q_n| $$


### Outras Métricas

Existem na literatura diversas métricas de similaridade, indicadas a diferentes tipos (ou grupos) de problemas. Entre as métricas comuns estão a medida de Similaridade por Cossenos, distâncias Minkowski, Chebyshev e Mahalanobis.


### Referências

* Wikipedia Contributors. *Distância Euclidiana*. 2021. Disponível em: https://pt.wikipedia.org/wiki/Dist%C3%A2ncia_euclidiana. Acesso em: 2021-12-27.

* Wikipedia Contributors. *Taxicab geometry*. 2021. Disponível em: https://en.wikipedia.org/wiki/Taxicab_geometry. Acesso em: 2021-12-27.

* Scikit-Learn. *DistanceMetric*. 2021. Disponível em: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.DistanceMetric.html#sklearn.metrics.DistanceMetric. Acesso em: 2021-12-27.

In [ ]:
model_knn=NearestNeighbors(metric="euclidean")
model_knn.fit(matriz_features)

distances, indices=model_knn.kneighbors(imoveis.iloc[0].values.reshape(1,-1),n_neighbors=10)

for i, d in zip(indices[0], distances[0]):
    print('Item: {} \t Distância: {:G}'.format(i, d))
print()

In [ ]:
filter_df  = imoveis[imoveis.index.isin(indices[0])]
filter_df

Os resultados obtidos podem ser comparados com valores aleatórios, a fim de verificar a qualidade.

In [ ]:
aleat= [0]
for i in range(1, 10):
    aleat.append(np.random.randint(0,1000))

filter_df  = imoveis[imoveis.index.isin(aleat)]
filter_df

#### Seleção de Imóveis com Preço Alto

O código abaixo seleciona imóveis de custo alto, para que se possa analisar as recomendações.

In [ ]:
hist = imoveis['price'].hist(bins=10)
hist

In [ ]:
imoveis[imoveis['price'] > 800000]

In [ ]:
#filtrando o imóvel 157 (deve retornar somente imóveis com muitos quartos e banheiros)
distances, indices=model_knn.kneighbors(imoveis.iloc[157].values.reshape(1,-1),n_neighbors=10)

filter_df  = imoveis[imoveis.index.isin(indices[0])]
filter_df

## Normalização (Opcional)

A normalização pode melhorar o desempenho do modelo em alguns casos, ao reduzir a faixa de valores possíveis. 

*Obs.: Essa é uma atividade que pode requerer um cuidado especial, uma vez que uma normalização mal feita pode não causar nenhum efeito ou até mesmo prejudicar o modelo.*

In [ ]:
scaler = MaxAbsScaler()
scaler.fit(imoveis)
scaled = scaler.transform(imoveis)
imoveis_scaled = pd.DataFrame(scaled, columns=imoveis.columns)

imoveis_scaled

In [ ]:
matriz_scaled = csr_matrix(imoveis_scaled.values)

model_knn=NearestNeighbors(metric="euclidean")
model_knn.fit(matriz_scaled)

distances, indices = model_knn.kneighbors(imoveis_scaled.iloc[0].values.reshape(1,-1),n_neighbors=20)

for i, d in zip(indices[0], distances[0]):
    print('Item: {} \t Distância: {:G}'.format(i, d))
print()

Os índices gerados serão utilizados para filtrar as informações originais, para facilitar o entendimento das informações.

In [ ]:
filter_sc = imoveis[imoveis.index.isin(indices[0])]
filter_sc

## Variáveis Categóricas, Ordinais e Textuais

Variáveis categóricas, textuais e ordinais devem passar tratamentos especiais, para que possam ser melhor utilizadas pelos algoritmos.

Como é utilizado um cálculo de similaridade, a definição entre textos pode causar incorreções.

*Ex.: Considere as cidades de Belo Horizonte, Betim e Contagem. Classificando lexicamente, Betim está mais perto de Belo Horizonte. No entanto, classificando geograficamente, Betim está mais próximo de Contagem. Logo, a métrica (distância léxica ou geográfica) influenciará no resultado final.*

Caso as variáveis não possam ser correlacionadas, deve ser feita a binarização dos valores. Estes devem ser colocados em formato *one-hot*, para que não influenciem um no outro.

No caso de imóveis, a informação de *imóvel para alugar* não é relacionada à *imóvel para comprar*, uma vez que os clientes para ambos os tipos de imóveis são diferentes (obs.: em outros contextos, pode existir uma correlação, mas não no exemplo). Com isso, o atributo de ALUGUEL/VENDA deve ser binarizado.

A representação do atributo de um imóvel para alugar ficaria, por exemplo, [0, 1], enquanto um imóvel para vender seria representado por [1, 0].

### Exemplo: Representação de Gêneros de Filmes

A imagem abaixo contém a representação de genêros de filmes.

![filmes](https://miro.medium.com/max/229/1*FWZeOK54Z_ubdpmQZoJqPA.png)


![filmes](https://miro.medium.com/max/511/1*8O1n4Zk1QTwk86IOoXbV5A.png)
*Fonte: https://www.analyticsvidhya.com/blog/2020/08/recommendation-system-k-nearest-neighbors/*


Mais informações e exemplos podem ser encontradas no link: https://www.analyticsvidhya.com/blog/2020/08/recommendation-system-k-nearest-neighbors/